In [1]:
import pandas as pd

chunksize = 10

In [2]:
filename = "meta_Clothing_Shoes_and_Jewelry.json"

iter = pd.read_json(filename, chunksize = chunksize, lines = True)

In [3]:
isFirstChunk = True

for chunk in iter:
    if isFirstChunk:
        print("First Chunk")
        print(chunk)
        chunk.to_json("Sample_Meta", orient='records', lines=True)
        sample_meta = chunk
        isFirstChunk = False
        break

First Chunk
                                            category  \
0  [Clothing, Shoes & Jewelry, Costumes & Accesso...   
1  [Clothing, Shoes & Jewelry, Luggage & Travel G...   
2  [Clothing, Shoes & Jewelry, Luggage & Travel G...   
3  [Clothing, Shoes & Jewelry, Luggage & Travel G...   
4  [Clothing, Shoes & Jewelry, Novelty & More, Cl...   
5  [Clothing, Shoes & Jewelry, Women, Clothing, S...   
6  [Clothing, Shoes & Jewelry, Women, Shoes, Sand...   
7  [Clothing, Shoes & Jewelry, Luggage & Travel G...   
8  [Clothing, Shoes & Jewelry, Luggage & Travel G...   
9  [Clothing, Shoes & Jewelry, Novelty & More, Cl...   

                                         description  \
0  [6" long, stretched waist measures 11 1/2" acr...   
1  [The Hottest Bag in Town! Brand: Anello Condit...   
2  [The Hottest Bag in Town! Brand: Anello Condit...   
3  [The Hottest Bag in Town! Brand: Anello Condit...   
4  [Brand New.  Hat Centre Length: adult about 8c...   
5  [Please allow 1-2cm dimension de

In [4]:
filename = 'Clothing_Shoes_and_Jewelry.json'

iter = pd.read_json(filename, chunksize = chunksize, lines = True)

In [5]:
isFirstChunk = True

for chunk in iter:
    if isFirstChunk:
        print("First Chunk")
        print(chunk)
        chunk.to_json("Sample_Review", orient='records', lines=True)
        sample_review = chunk
        isFirstChunk = False
        break

First Chunk
   overall  vote  verified   reviewTime      reviewerID       asin  \
0        5   2.0      True   05 4, 2014  A2IC3NZN488KWK  871167042   
1        4   NaN      True  04 26, 2014  A3OT9BYASFGU2X  871167042   
2        5   6.0     False  04 17, 2014  A28GK1G2KDXHRP  871167042   
3        5   NaN      True  04 16, 2014  A3NFXFEKW8OK0E  871167042   
4        5   NaN      True  04 15, 2014  A3I6G5TKBVJEK9  871167042   
5        5   NaN      True  03 31, 2014  A1A7Y1M8AJWNZ8  871167042   
6        5   NaN      True  03 31, 2014  A30FG02C424EJ5  871167042   
7        5   NaN      True  03 28, 2014   ADQQYU1UCDEWB  871167042   
8        5  14.0      True  03 23, 2014  A39YL2NXZORK56  871167042   
9        5   7.0      True  03 20, 2014  A2PRY50ZESF1MH  871167042   

                       style   reviewerName  \
0  {'Format:': ' Paperback'}     Ruby Tulip   
1  {'Format:': ' Paperback'}      Laurie K.   
2  {'Format:': ' Paperback'}  Marie Rhoades   
3  {'Format:': ' Paperback'} 

In [6]:
def find_noun_head(token):
    #print(f"Finding Token: {token}")
    #print(f"Token Head: {token.head}")
    if token.head.pos_ == "NOUN" or token.head.dep_ == "ROOT":
        #print("Found")
        return token.head
    else:
        #print("Continue Search")
        return find_noun_head(token.head)
    
def find_verb_head(token):
    #print(f"Finding Token: {token}")
    #print(f"Token Head: {token.head}")
    if token.head.pos_ == "VERB" or token.head.dep_ == "ROOT":
        #print("Found")
        return token.head
    else:
        #print("Continue Search")
        return find_noun_head(token.head)

In [7]:
#https://towardsdatascience.com/aspect-based-sentiment-analysis-using-spacy-textblob-4c8de3e0d2b9
#https://pythonprogramming.net/sentiment-analysis-python-textblob-vader/
#https://spacy.io/usage/linguistic-features

#Next Steps:
#Negation, admodal, intensifier, etc
#Passive voice

from textblob import TextBlob

import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_sm")

recommendation_verb = ['recommend', 'discourage', 'critize', 'like', 'dislike', 'love']

for index in range(chunksize):
    review = str(sample_review.iloc[index]["reviewText"])
    print(f"Review Text: {review}")
    
    review_analysis = TextBlob(review)
    print(review_analysis.sentiment)
    print("")
    
    count = 0
    doc = nlp(review)
    for sent in doc.sents:
        if sent.text.strip() != "":
            count += 1
            print(f"Sentence {count}: {sent}")

            sent_analysis = TextBlob(sent.text)
            print(sent_analysis.sentiment)

            noun_descriptor_pair = []
            noun_verb_pair = []
            adjectives = [tok for tok in sent if tok.pos_ == "ADJ"]
            noun_obj = [tok for tok in sent if tok.pos_ == "NOUN" and tok.dep_ == "dobj"]
            
            for adjective in adjectives:
                #print(f"Intial Adjective: {adjective}")
                descriptor = ""
                for child in adjective.children:
                    if child.pos_ == "ADV":
                        descriptor += child.text + " "
                        print(descriptor)
                descriptor += adjective.text
                    
                noun = find_noun_head(adjective)
                if noun.pos_ == "NOUN":
                    negation_tokens = []
                    amodal_tokens = []
                    children = [tok for tok in noun.children]
                    for child in children:
                        if child.dep_ == "neg":
                            negation_tokens.append(child)
                        if child.dep == "amod":
                            amodal_tokens.append(child)

                    #print(f"({descriptor}, {noun})")
                    noun_descriptor_pair.append((descriptor, noun))
                                        
            for noun in noun_obj:
                #print(f"Intial Verb: {verb}")
                verb = find_verb_head(noun)
                    
                if verb.pos_ == "VERB" and verb.text in recommendation_verb:
                    #print(f"({verb}, {noun})")
                    noun_verb_pair.append((verb, noun))

            displacy.render(sent, style="dep")
            print(f"Adjectives: {adjectives}")
            print(f"noun_descriptor_pair: {noun_descriptor_pair}")
            print(f"Nouns: {noun_obj}")
            print(f"noun_verb_pair: {noun_verb_pair}")            
            print("")

    print("---------------------------------------------------------------")
    print("")

Review Text: This book has beautiful photos, good and understandable directions, and many different kinds of jewelry.  Wire working and metalsmithing jewelry are covered.  Highly recommend this book.
Sentiment(polarity=0.442, subjectivity=0.648)

Sentence 1: This book has beautiful photos, good and understandable directions, and many different kinds of jewelry.
Sentiment(polarity=0.5125, subjectivity=0.675)


Adjectives: [beautiful, good, understandable, many, different]
noun_descriptor_pair: [('beautiful', photos), ('good', directions), ('understandable', directions), ('many', kinds), ('different', kinds)]
Nouns: [photos]
noun_verb_pair: []

Sentence 2: Wire working and metalsmithing jewelry are covered.
Sentiment(polarity=0.0, subjectivity=0.0)


Adjectives: []
noun_descriptor_pair: []
Nouns: [jewelry]
noun_verb_pair: []

Sentence 3: Highly recommend this book.
Sentiment(polarity=0.16, subjectivity=0.5399999999999999)


Adjectives: []
noun_descriptor_pair: []
Nouns: [book]
noun_verb_pair: [(recommend, book)]

---------------------------------------------------------------

Review Text: I love the ideas but have no access sea glass, but will experiment with other stones ... but I do love the book.  Great job!
Sentiment(polarity=0.46875, subjectivity=0.58125)

Sentence 1: I love the ideas but have no access sea glass, but will experiment with other stones ...
Sentiment(polarity=0.1875, subjectivity=0.4875)


Adjectives: [other]
noun_descriptor_pair: [('other', stones)]
Nouns: [ideas, glass]
noun_verb_pair: [(love, ideas)]

Sentence 2: but I do love the book.
Sentiment(polarity=0.5, subjectivity=0.6)


...

Adjectives: [beautiful, new]
noun_descriptor_pair: [('new', chance)]
Nouns: [chance]
noun_verb_pair: []

Sentence 8: Great purchase!
Sentiment(polarity=1.0, subjectivity=0.75)


Adjectives: [Great]
noun_descriptor_pair: [('Great', purchase)]
Nouns: []
noun_verb_pair: []

---------------------------------------------------------------

Review Text: This beautifully photographed book provides step-by-step instruction from beginner level to more advanced projects in hand-crafted jewelry using sea glass, stones, and crystals. Having admired and worn Beth Martin's jewelry crafted from sea glass found along the shores of Lake Erie, I thoroughly enjoyed reading about the materials, tools, and skills that combine to make stunning, one-of-a-kind wearable art. This book has everything the beginner needs to produce lovely pieces, and, with 50 different projects, it offers ideas and inspiration for the seasoned and skilled jewelry designer as well.
Sentiment(polarity=0.41, subjectivity=0.59)

Sentence 1: This beautifully photographed book provides step-by-step instruction from beginner level to more advanced projects in hand-crafted jewelry using sea glass, stones, and cr

Adjectives: [advanced]
noun_descriptor_pair: [('more advanced', projects)]
Nouns: [instruction, glass]
noun_verb_pair: []

Sentence 2: Having admired and worn Beth Martin's jewelry crafted from sea glass found along the shores of Lake Erie, I thoroughly enjoyed reading about the materials, tools, and skills that combine to make stunning, one-of-a-kind wearable art.
Sentiment(polarity=0.5, subjectivity=0.85)


Adjectives: [stunning, wearable]
noun_descriptor_pair: [('stunning', art), ('wearable', art)]
Nouns: [jewelry, art]
noun_verb_pair: []

Sentence 3: This book has everything the beginner needs to produce lovely pieces, and, with 50 different projects, it offers ideas and inspiration for the seasoned and skilled jewelry designer as well.
Sentiment(polarity=0.27, subjectivity=0.42000000000000004)


Adjectives: [lovely, different, seasoned, skilled]
noun_descriptor_pair: [('lovely', pieces), ('different', projects), ('seasoned', designer), ('skilled', designer)]
Nouns: [pieces, ideas]
noun_verb_pair: []

---------------------------------------------------------------



In [8]:
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_sm")

doc = nlp("The restaurant was expensive, but the menu was great")
for sent in doc.sents:
    if sent.text.strip() != "":
        count += 1
        print(f"Sentence {count}: {sent}")

        sent_analysis = TextBlob(sent.text)
        print(sent_analysis.sentiment)

        noun_descriptor_pair = []
        noun_verb_pair = []
        adjectives = [tok for tok in sent if tok.pos_ == "ADJ"]
        noun_obj = [tok for tok in sent if tok.pos_ == "NOUN" and tok.dep_ == "dobj"]

        for adjective in adjectives:
            #print(f"Intial Adjective: {adjective}")
            descriptor = ""
            for child in adjective.children:
                if child.pos_ == "ADV":
                    descriptor += child.text + " "
                    print(descriptor)
            descriptor += adjective.text

            noun = find_noun_head(adjective)
            if noun.pos_ == "NOUN":
                negation_tokens = []
                amodal_tokens = []
                children = [tok for tok in noun.children]
                for child in children:
                    if child.dep_ == "neg":
                        negation_tokens.append(child)
                    if child.dep == "amod":
                        amodal_tokens.append(child)

                #print(f"({descriptor}, {noun})")
                noun_descriptor_pair.append((descriptor, noun))

        for noun in noun_obj:
            #print(f"Intial Verb: {verb}")
            verb = find_verb_head(noun)

            if verb.pos_ == "VERB" and verb.text in recommendation_verb:
                #print(f"({verb}, {noun})")
                noun_verb_pair.append((verb, noun))

        displacy.render(sent, style="dep")
        print(f"Adjectives: {adjectives}")
        print(f"noun_descriptor_pair: {noun_descriptor_pair}")
        print(f"Nouns: {noun_obj}")
        print(f"noun_verb_pair: {noun_verb_pair}")            
        print("")

print("---------------------------------------------------------------")
print("")

Sentence 4: The restaurant was expensive, but the menu was great
Sentiment(polarity=0.15000000000000002, subjectivity=0.725)


Adjectives: [expensive, great]
noun_descriptor_pair: []
Nouns: []
noun_verb_pair: []

---------------------------------------------------------------

